# About This Project

In this project, our artificial intelligence models predict the results of football matches and the winning probabilities of the teams. Predictions are purely probabilistic predictions and may not reflect the actual result.

***The project can only predict league matches!***

The project was trained by giving data from the 2nd week of the league matches.
You can check out my github page for detailed information.
(https://github.com/55utd55/Oracle-Octopus-Ai)

-----

Check out my YouTube videos below for installation and usage of the project!

English: https://www.youtube.com/watch?v=TNc-YB2-I_g

Turkish: https://www.youtube.com/watch?v=mGy0rjbiXFE

In [ ]:
!pip install tensorflow==2.14.0

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import requests
from bs4 import BeautifulSoup
import json
import shutil
import os
from datetime import datetime
import zipfile
from huggingface_hub import hf_hub_download

# Download Models

In [ ]:
local_filepath = hf_hub_download(repo_id="55utd55/oracle-octopus-ai", filename="mssonuc-models.zip")

In [ ]:
with zipfile.ZipFile(local_filepath, 'r') as zip_ref:
    zip_ref.extractall("/content/mssonuc_models")

In [ ]:
csv_local_filepath = hf_hub_download(repo_id="55utd55/oracle-octopus-ai", filename="biggest-data.csv")

# Get Data on Week Matches
https://arsiv.mackolik.com/Canli-Sonuclar

In [ ]:
os.mkdir("/content/csv/")
os.mkdir("/content/csv/model-results/")

In [ ]:
orjcsv = csv_local_filepath
copycsv = "/content/csv/cs.csv"
shutil.copyfile(orjcsv, copycsv)
csvget = pd.read_csv(copycsv, skipinitialspace=True, na_values="?", comment="\t", sep=",")

In [ ]:
print("If you want to stop data collection, type: stop")
while 1 > 0:
    ligid = input()
    ligyili = datetime.now().year
    ligadi = str(ligid)
    if str(ligid) == "stop":
        csvget.to_csv("/content/csv/cs.csv", index=False)
        print("Stopped!")
        break
    headers = {'Accept': 'application/json'}
    ligsonhaftaverisi = requests.get('https://arsiv.mackolik.com/AjaxHandlers/StandingHandler.ashx?op=standing&id='+str(ligid), headers=headers)
    ligsonhaftaverisi = ligsonhaftaverisi.json()
    for maclar in ligsonhaftaverisi["f"]:
        try:
            evgol = 0
            depgol = 0
            evid = maclar[3]
            depid = maclar[4]
            evsira = 0
            depsira = 0
            for siralamalar in ligsonhaftaverisi["s"]:
                try:
                    takimid = siralamalar[0]
                    evsira += 1
                    depsira += 1
                    if takimid == evid:
                        evyuzdelikdilim = evsira * 100 / len(ligsonhaftaverisi["s"])
                        evims = siralamalar[2]
                        evdms = siralamalar[3]
                        evig = siralamalar[4]
                        evdg = siralamalar[5]
                        evib = siralamalar[6]
                        evdb = siralamalar[7]
                        evim = siralamalar[8]
                        evdm = siralamalar[9]
                        eviag = siralamalar[10]
                        evdag = siralamalar[11]
                        eviyg = siralamalar[12]
                        evdyg = siralamalar[13]
                        evip = siralamalar[14]
                        evdp = siralamalar[15]
                    if takimid == depid:
                        depyuzdelikdilim = depsira * 100 / len(ligsonhaftaverisi["s"])
                        depims = siralamalar[2]
                        depdms = siralamalar[3]
                        depig = siralamalar[4]
                        depdg = siralamalar[5]
                        depib = siralamalar[6]
                        depdb = siralamalar[7]
                        depim = siralamalar[8]
                        depdm = siralamalar[9]
                        depiag = siralamalar[10]
                        depdag = siralamalar[11]
                        depiyg = siralamalar[12]
                        depdyg = siralamalar[13]
                        depip = siralamalar[14]
                        depdp = siralamalar[15]
                except:
                    continue
            if evgol > depgol:
                mssonuc = 0
            if evgol == depgol:
                mssonuc = 1
            if evgol > depgol:
                mssonuc = 2
            ekleneceksatir = {"ligadi": str(ligadi), "ligyili": str(ligyili), "evid": str(evid), "depid": str(depid), "evyuzdelikdilim": str(evyuzdelikdilim), "depyuzdelikdilim": str(depyuzdelikdilim), "evims": str(evims), "depims": str(depims), "evdms": str(evdms), "depdms": str(depdms), "evig": str(evig), "depig": str(depig), "evdg": str(evdg), "depdg": str(depdg), "evib": str(evib), "depib": str(depib), "evdb": str(evdb), "depdb": str(depdb), "evim": str(evim), "depim": str(depim), "evdm": str(evdm), "depdm": str(depdm), "eviag": str(eviag), "depiag": str(depiag), "evdag": str(evdag), "depdag": str(depdag), "eviyg": str(eviyg), "depiyg": str(depiyg), "evdyg": str(evdyg), "depdyg": str(depdyg), "evip": str(evip), "depip": str(depip), "evdp": str(evdp), "depdp": str(depdp), "evgol": str(evgol), "depgol": str(depgol)}
            csvget = pd.concat([csvget, pd.DataFrame([ekleneceksatir])], ignore_index=True)
            print("Match Added!")
        except Exception as e:
            print(e)

# Load Models

In [ ]:
modelbir = load_model("/content/mssonuc_models/gol-tahmin-big-data-v2")
modeliki = load_model("/content/mssonuc_models/models-beta-uclu-model-v4")
modeluc = load_model("/content/mssonuc_models/gol-tahmin-v2-ve-mssonuc-beta-v2-standartsapma")
dnn_model = load_model("/content/mssonuc_models/evgoltahmin-v1")
depdnn_model = load_model("/content/mssonuc_models/depgoltahmin-v1")

# Home and Away Goal Count Prediction v1

In [ ]:
adres = "/content/csv/cs.csv"
veriseti = pd.read_csv(adres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
veriseti = veriseti.dropna()
veriseti = veriseti.reset_index(drop=True)
verisetimacadli = veriseti
veriseti = veriseti[['evyuzdelikdilim',
       'depyuzdelikdilim', 'evims', 'depims', 'evdms', 'depdms', 'evig',
       'depig', 'evdg', 'depdg', 'evib', 'depib', 'evdb', 'depdb', 'evim',
       'depim', 'evdm', 'depdm', 'eviag', 'depiag', 'evdag', 'depdag', 'eviyg',
       'depiyg', 'evdyg', 'depdyg', 'evip', 'depip', 'evdp', 'depdp']]
veriseti.isna().sum()
veriseti = veriseti.dropna()
x = veriseti.reset_index(drop=True)
predictions = modelbir.predict(np.asarray(x))
prediction_list = predictions.tolist()
dfcikti = pd.DataFrame(predictions, columns=['evgoltahmin', 'depgoltahmin'])
sontablo = pd.merge(verisetimacadli, dfcikti, left_index=True, right_index=True)
sontablo.to_csv("/content/csv/model-results/ms1.csv", index=False)

# Match Result Prediction v1

In [ ]:
adres = "/content/csv/cs.csv"
veriseti = pd.read_csv(adres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
veriseti.isna().sum()
veriseti = veriseti.dropna()
veriseti = veriseti.reset_index(drop=True)
veriseti = veriseti[['evyuzdelikdilim',
       'depyuzdelikdilim', 'evims', 'depims', 'evdms', 'depdms', 'evig',
       'depig', 'evdg', 'depdg', 'evib', 'depib', 'evdb', 'depdb', 'evim',
       'depim', 'evdm', 'depdm', 'eviag', 'depiag', 'evdag', 'depdag', 'eviyg',
       'depiyg', 'evdyg', 'depdyg', 'evip', 'depip', 'evdp', 'depdp']]
predictions = modeliki.predict(np.asarray(veriseti))
dfcikti = pd.DataFrame(predictions, columns=['evolasilik', 'berolasilik', 'depolasilik'])
sontablo = pd.merge(verisetimacadli, dfcikti, left_index=True, right_index=True)
sontablo.to_csv("/content/csv/model-results/mssonuc-beta-gol-tahmin-version.csv", index=False)

# Match Result Prediction v2

In [ ]:
ms1adres = "/content/csv/model-results/ms1.csv"
ms1veriseti = pd.read_csv(ms1adres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
ms1veriseti = ms1veriseti[['ligadi', 'ligyili', 'evid', 'depid', 'evyuzdelikdilim',
       'depyuzdelikdilim', 'evims', 'depims', 'evdms', 'depdms', 'evig',
       'depig', 'evdg', 'depdg', 'evib', 'depib', 'evdb', 'depdb', 'evim',
       'depim', 'evdm', 'depdm', 'eviag', 'depiag', 'evdag', 'depdag', 'eviyg',
       'depiyg', 'evdyg', 'depdyg', 'evip', 'depip', 'evdp', 'depdp', 'evgol',
       'depgol', 'evgoltahmin', 'depgoltahmin']]
msbetaadres = "/content/csv/model-results/mssonuc-beta-gol-tahmin-version.csv"
msbetaveriseti = pd.read_csv(msbetaadres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
msbetaveriseti = msbetaveriseti.loc[:, ["evolasilik", "berolasilik", "depolasilik"]]
ms1veriseti = ms1veriseti.reset_index(drop=True)
msbetaveriseti = msbetaveriseti.reset_index(drop=True)
veriseti = pd.merge(ms1veriseti, msbetaveriseti, left_index=True, right_index=True)
gercekveriseti = veriseti
veriseti = veriseti[['evyuzdelikdilim',
       'depyuzdelikdilim', 'evims', 'depims', 'evdms', 'depdms', 'evig',
       'depig', 'evdg', 'depdg', 'evib', 'depib', 'evdb', 'depdb', 'evim',
       'depim', 'evdm', 'depdm', 'eviag', 'depiag', 'evdag', 'depdag', 'eviyg',
       'depiyg', 'evdyg', 'depdyg', 'evip', 'depip', 'evdp', 'depdp', 'evgoltahmin', 'depgoltahmin', 'evolasilik',
       'berolasilik', 'depolasilik']]
veriseti.isna().sum()
veriseti = veriseti.dropna()
veriseti = np.array(veriseti)
means = np.mean(veriseti, axis=0)
stds = np.std(veriseti, axis=0)
veriseti = (veriseti - means) / stds
predictions = modeluc.predict(veriseti)
predictions = np.array(predictions)
stdsapma = np.std(predictions, axis=1)
dfcikti = pd.DataFrame(predictions, columns=['lastevolasilik', 'lastberolasilik', 'lastdepolasilik'])
dfstdsapma = pd.DataFrame(stdsapma, columns=['stdsapma'])
stdtablo = pd.merge(gercekveriseti, dfstdsapma, left_index=True, right_index=True)
sontablo = pd.merge(stdtablo, dfcikti, left_index=True, right_index=True)
sontablo.to_csv("/content/csv/model-results/last.csv", index=False)
ms1adres = "/content/csv/model-results/last.csv"
sontablo = pd.read_csv(ms1adres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
sontablo["lastevolasilik"] -= sontablo["stdsapma"]
sontablo["lastberolasilik"] -= sontablo["stdsapma"]
sontablo["lastdepolasilik"] -= sontablo["stdsapma"]
sontablo.to_csv("/content/csv/model-results/last-sapmali-version.csv", index=False)
evgoltahminveriseti = sontablo
evgoltahminveriseti = evgoltahminveriseti.reset_index(drop=True)
evgoltahminveriseti.to_csv("/content/csv/model-results/evgoltahminveriseti.csv", index=False)
evgoltahminveriseti.to_csv("/content/csv/model-results/depgoltahminveriseti.csv", index=False)

# Home Team Goals Prediction v1

In [ ]:
adres = "/content/csv/model-results/evgoltahminveriseti.csv"
veriseti = pd.read_csv(adres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
veriseti.isna().sum()
veriseti = veriseti.dropna()
veriseti['evgol'] = veriseti['evgol'].where(veriseti['evgol'] < 4, 4)
df = veriseti
veriseti = veriseti[['evyuzdelikdilim', 'depyuzdelikdilim', 'evims',
       'depims', 'evdms', 'depdms', 'evig', 'depig', 'evdg', 'depdg', 'evib',
       'depib', 'evdb', 'depdb', 'evim', 'depim', 'evdm', 'depdm', 'eviag',
       'depiag', 'evdag', 'depdag', 'eviyg', 'depiyg', 'evdyg', 'depdyg',
       'evip', 'depip', 'evdp', 'depdp']]
veriseti.isna().sum()
veriseti = veriseti.dropna()
predictions = dnn_model.predict(veriseti)
predictions_df = pd.DataFrame(predictions, columns=['evclass1_prob', 'evclass2_prob', 'evclass3_prob', 'evclass4_prob', 'evclass5_prob'])
df = pd.concat([df, predictions_df], axis=1)
df.to_csv("/content/csv/model-results/evgoltahminveriseti-tahmin-version.csv", index=False)
predictions = np.array(predictions)
stdsapma = np.std(predictions, axis=1)
dfstdsapma = pd.DataFrame(stdsapma, columns=['stdsapma_class'])
stdtablo = pd.merge(df, dfstdsapma, left_index=True, right_index=True)
stdtablo.to_csv("/content/csv/model-results/evgoltahminveriseti-tahmin-version-last.csv", index=False)

# Away Team Goals Prediction v1

In [ ]:
adres = "/content/csv/model-results/depgoltahminveriseti.csv"
veriseti = pd.read_csv(adres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
veriseti.isna().sum()
veriseti = veriseti.dropna()
veriseti['depgol'] = veriseti['depgol'].where(veriseti['depgol'] < 4, 4)
df = veriseti
veriseti = veriseti[['evyuzdelikdilim', 'depyuzdelikdilim', 'evims',
       'depims', 'evdms', 'depdms', 'evig', 'depig', 'evdg', 'depdg', 'evib',
       'depib', 'evdb', 'depdb', 'evim', 'depim', 'evdm', 'depdm', 'eviag',
       'depiag', 'evdag', 'depdag', 'eviyg', 'depiyg', 'evdyg', 'depdyg',
       'evip', 'depip', 'evdp', 'depdp']]
veriseti.isna().sum()
veriseti = veriseti.dropna()
predictions = depdnn_model.predict(veriseti)
predictions_df = pd.DataFrame(predictions, columns=['depclass1_prob', 'depclass2_prob', 'depclass3_prob', 'depclass4_prob', 'depclass5_prob'])
df = pd.concat([df, predictions_df], axis=1)
df.to_csv("/content/csv/model-results/depgoltahminveriseti-tahmin-version.csv", index=False)
predictions = np.array(predictions)
stdsapma = np.std(predictions, axis=1)
dfstdsapma = pd.DataFrame(stdsapma, columns=['depstdsapma_class'])
stdtablo = pd.merge(df, dfstdsapma, left_index=True, right_index=True)
stdtablo.to_csv("/content/csv/model-results/depgoltahminveriseti-tahmin-version-last.csv", index=False)
evgoltahminverisetiadres = "/content/csv/model-results/evgoltahminveriseti-tahmin-version-last.csv"
evgoltahminveriseti = pd.read_csv(evgoltahminverisetiadres, skipinitialspace=True, na_values="?", comment="\t", sep=",", low_memory=False)
deptablo = stdtablo.loc[:, ["depclass1_prob", "depclass2_prob", "depclass3_prob", "depclass4_prob", "depclass5_prob", "depstdsapma_class"]]
sontablo = pd.concat([evgoltahminveriseti, deptablo], axis=1)
sontablo.to_csv("/content/csv/model-results/yari-sontablo.csv", index=False)
sontablo = sontablo[112365:].reset_index()
sontablo["evurl"] = "https://arsiv.mackolik.com/Takim/" + sontablo["evid"].astype(str) + "/"
sontablo["depurl"] = "https://arsiv.mackolik.com/Takim/" + sontablo["depid"].astype(str) + "/"
for i, row in sontablo.iterrows():
    evurl = sontablo.iloc[i]['evurl']
    html_content = requests.get(str(evurl)).content
    soup = BeautifulSoup(html_content, 'html.parser')
    hometeamname = soup.find_all('h1')[0].get_text()
    depurl = sontablo.iloc[i]['depurl']
    dep_html_content = requests.get(str(depurl)).content
    soup = BeautifulSoup(dep_html_content, 'html.parser')
    awayteamname = soup.find_all('h1')[0].get_text()
    match_name = str(hometeamname)+" - "+str(awayteamname)
    sontablo.loc[i, "match_name"] = match_name
final = sontablo.loc[:, ['match_name', 'evolasilik', 'berolasilik', 'depolasilik',
       'evclass1_prob', 'evclass2_prob', 'evclass3_prob', 'evclass4_prob',
       'evclass5_prob', 'depclass1_prob', 'depclass2_prob', 'depclass3_prob',
       'depclass4_prob', 'depclass5_prob', 'evurl', 'depurl']]
final.columns = ['match_name', 'home_team_probability', 'draw_probability', 'away_team_probability',
       'probability_of_home_team_scoring_zero_goal', 'probability_of_home_team_scoring_one_goal', 'probability_of_home_team_scoring_two_goals', 'probability_of_home_team_scoring_three_goals',
       'probability_of_home_team_scoring_four_and_upper_than_four_goals', 'probability_of_away_team_scoring_zero_goal', 'probability_of_away_team_scoring_one_goal', 'probability_of_away_team_scoring_two_goals', 'probability_of_away_team_scoring_three_goals',
       'probability_of_away_team_scoring_four_and_upper_than_four_goals', 'home_team_page_url', 'away_team_page_url']
final.to_csv("/content/csv/model-results/final.csv", index=False)
for file_name in os.listdir("/content/csv/model-results/"):
    if file_name != "final.csv":
        os.remove(os.path.join("/content/csv/model-results/", file_name))
print("Predictions are saved in the csv/model-results/final.csv directory.")

# Output
If you have come this far, the predictions of models will be saved in the csv/model/final.csv file. You can still see the predictions with the code below.

In [ ]:
final